In [1]:
import torch
from torch import nn
!pip install torchtext 

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
import pandas as pd
from torchtext.data import Field

In [4]:
from torchtext.vocab import GloVe
glove_embeddings=GloVe(name='6B',dim='50')

In [5]:
import string
all_letters = string.ascii_letters + " .,;'-"
print(all_letters)

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'-


In [3]:
# inp=torch.rand(20,16,50)
# word=['w','o','r','d']
# char=nn.Conv1d(16,100,7)
# out=char(inp)
# print(out.shape)

torch.Size([20, 100, 44])


In [6]:
df_train=pd.read_csv('switchboard_train.csv')
print(df_train.head())
text_field = Field(
    tokenize='basic_english', 
    lower=True)
label_field = Field(sequential=False, use_vocab=True)
preprocessed_text = df_train['clean_text'].apply(lambda x: text_field.preprocess(str(x)))
text_field.build_vocab(
    preprocessed_text, 
    vectors='glove.6B.50d')
# vocab=text_field.vocab

              swda_filename ptb_basename  conversation_no  transcript_index  \
0  sw00utt/sw_0001_4325.utt     4/sw4325             4325                 0   
1  sw00utt/sw_0001_4325.utt     4/sw4325             4325                 1   
2  sw00utt/sw_0001_4325.utt     4/sw4325             4325                 2   
3  sw00utt/sw_0001_4325.utt     4/sw4325             4325                 3   
4  sw00utt/sw_0001_4325.utt     4/sw4325             4325                 4   

  act_tag                      act_label_1 act_label_2 act_label_relation  \
0       o                            Other        none               none   
1      qw         Info-request:Wh-Question        none               none   
2    qy^d     Info-request:Yes-No-Question        none               none   
3       +  Other:Segment-(multi-utterance)        none               none   
4       +  Other:Segment-(multi-utterance)        none               none   

  caller utterance_index subutterance_index  \
0      A       

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [9]:
df=pd.read_csv('switchboard_complete.csv')
df_train=df[:160000]
df_valid=df[160000:200000]
df_test=df[200000:]
df_train=df_train.fillna('')
df_valid=df_valid.fillna('')
df_test=df_test.fillna('')

In [10]:
# df_train.to_csv('switchboard_train.csv',index=False)
# df_valid.to_csv('switchboard_valid.csv',index=False)
df_test.to_csv('switchboard_test.csv',index=False)

In [7]:
from torchtext.data import TabularDataset
train_datafields=[('swda_filename',None),('ptb_basename',None),('conversation_no',None),('transcript_index',None),('act_tag',None),('act_label_1',label_field),('act_label_2',None),('act_label_relation',None),('caller',None),('utterance_index',None),('subutterance_index',None),('clean_text',text_field),('topic_description',None),('prompt',None)]
train,validation,test=TabularDataset.splits(path='',train='switchboard_train.csv',validation='switchboard_valid.csv',test='switchboard_test.csv',format='csv',skip_header=True,fields=train_datafields)

/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [8]:
train[0].__dict__.keys()
print(test[0].__dict__.keys())

dict_keys(['act_label_1', 'clean_text'])


In [9]:
print(len(train))

160000


In [10]:
text_field.build_vocab(train,vectors='glove.6B.50d')
label_field.build_vocab(train,vectors='glove.6B.50d')
# text_field.vocab.stoi['are']

In [11]:
word_vocabulary=text_field.vocab

In [12]:
print(len(word_vocabulary.vectors))
print(len(label_field.vocab))

18618
55


In [13]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# BATCH_SIZE = 128
batch_size=80


from torchtext.data import Iterator, BucketIterator

train_iter, val_iter = BucketIterator.splits((train, validation),batch_sizes=(batch_size,batch_size),device=device,sort_key=lambda x: len(x.clean_text),sort_within_batch=False,repeat=False)
test_iter = Iterator(test, batch_size=batch_size, device=device, sort=False, sort_within_batch=False, repeat=False)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: Iterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [14]:
class Attention(nn.Module):
    def __init__(self,da,u_2,r,k): #keep r and u_2 same
        super(Attention,self).__init__()
        self.ws1=torch.nn.Linear(u_2,da,bias=False)
        self.ws3=torch.nn.Linear(k,da,bias=True)
#         self.tanh=torch.tanh()
        self.ws2=torch.nn.Linear(da,r,bias=False)
        self.projection=nn.Linear(batch_size,50)
    def forward(self,utt_hid,conv_hid):
        lin1=self.ws1(utt_hid)
        lin2=self.ws3(conv_hid)
        add=torch.add(lin1,lin2)
        squash=torch.tanh(add)
        s=self.ws2(squash)
        attention=torch.softmax(s,dim=1)
#         print(attention.shape)
        scores=torch.matmul(attention,utt_hid.T)
        scores=self.projection(scores).flatten()
        return scores
    
class ConversationRNN(nn.Module):
    def __init__(self,embedding_dim,hidden_dim,num_layers):
        super(ConversationRNN,self).__init__()
        self.gru=nn.GRU(embedding_dim,hidden_size=hidden_dim,num_layers=num_layers,bidirectional=True)
    def forward(self,x):
        out,hid=self.gru(x)
        forward_hid=hid.view(self.gru.num_layers,2,x.shape[1],self.gru.hidden_size)[0,0,0]
        return forward_hid,out,hid
    
class UtteranceRNN(nn.Module):
    def __init__(self,embedding,embedding_dim=50,hidden_dim=100,num_layers=1):
        super(UtteranceRNN,self).__init__()
        self.embedding=embedding
        self.gru=nn.GRU(embedding_dim,hidden_dim,num_layers,bidirectional=True)
    def forward(self,x):
        embed=self.embedding[x]
        out,hid=self.gru(embed)
        return hid.view(x.shape[1],-1)
        
        
        

In [15]:
import numpy as np
def get_accuracy(true_labels,predictions):
    classes=torch.argmax(predictions,dim=1)
    return torch.mean((classes==true_labels).float())

In [16]:
!nvidia-smi

Thu Nov  5 11:39:21 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 450.36.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro M4000        On   | 00000000:00:05.0 Off |                  N/A |
| 46%   30C    P8    11W / 120W |      4MiB /  8126MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
i=0
batch_size=80
# uttRNN=UtteranceRNN(word_vocabulary.vectors,50,100,1)
# hid2labels=torch.nn.Linear(200,55)
loss_fn=nn.CrossEntropyLoss()
# att=Attention(da=300,u_2=200,r=200,k=100)
# convRNN=ConversationRNN(batch_size*50,batch_size*100,1)
params=list(uttRNN.parameters())+list(att.parameters())+list(convRNN.parameters())
epochs=2
optimizer=torch.optim.Adam(params)
for epoch in range(epochs):
    epoch_losses=[]
    epoch_acc=[]
    for idx,batch in enumerate(train_iter):
        conv_hid=torch.randn(1,batch_size*100)
        optimizer.zero_grad()
        utt_hid=uttRNN(batch.clean_text)
        scores=att(utt_hid,conv_hid.view(batch_size,-1))
        conv_hid,out,hid=convRNN(scores.view(1,1,-1))
        predicted_labels=hid2labels(hid.view(batch_size,-1)).to(device)
        accuracy=get_accuracy(batch.act_label_1,predicted_labels)
        epoch_acc.append(accuracy)
        loss=loss_fn(predicted_labels,batch.act_label_1)
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        if(idx%100==0):
            print('loss on epoch_no. {} and batch_no {} is {} and acc is {}'.format(epoch,idx,np.mean(epoch_losses),accuracy))
        
#     val_losses=[]
#     val_acc=[]
#     for idx,batch in enumerate(val_iter):
#         with torch.no_grad():
#             optimizer.zero_grad()
#             uttRNN.eval()
#             att.eval()
#             convRNN.eval()
#             utt_hid=uttRNN(batch.clean_text)
#             scores=att(utt_hid,conv_hid.view(batch_size,-1))
#             conv_hid,out,hid=convRNN(scores.view(1,1,-1))
#             labels=hid2labels(hid.view(batch_size,-1)).to(device)
#             t_loss=loss_fn(labels,batch.act_label_1)
#             val_accuracy=get_accuracy(batch.act_label_1,labels)
#             val_acc.append(val_accuracy)
#             val_losses.append(t_loss.item())
#             if(idx%100==0):
#                 print('test loss on epoch_no. {} and batch_ no {} is {} and val acc is {}'.format(epoch,idx,np.mean(val_losses),val_accuracy))
            
            
        
        
# embeddinggggg=word_vocabulary.vectors
# lstm=UtteranceRNN(word_vocabulary.vectors,50,100,1)
# for batch in train_iter:
#     i+=1
#     if(i<5):
# #         batch.clean_text=batch.clean_text.view(64,-1)
#         print(batch.clean_text.shape)
# #         embed=embeddinggggg[batch.clean_text]
#         hid=lstm(batch.clean_text)
# #         pr1int(hid)
#         print(hid.shape)
#         scores=att(hid.view(160,-1),conv_hidden.view(160,-1))
#         print(scores.shape)
#         conv_hidden,out=convRNN(scores.view(1,1,-1))
        
# #         for j in range(batch.clean_text.shape[0]):
# #             sent=batch.clean_text[j]
# #             for word in sent:ṇ
# #                 w=word_vocabulary.itos[word]
# # #                 print(type(w))
# #                 for char in w:
# #                     print(all_letters[char])
                    
        

loss on epoch_no. 0 and batch_no 0 is 2.722581148147583 and acc is 0.4000000059604645
loss on epoch_no. 0 and batch_no 100 is 2.7997438293872494 and acc is 0.32500001788139343
loss on epoch_no. 0 and batch_no 200 is 2.8046348094940186 and acc is 0.3125
loss on epoch_no. 0 and batch_no 300 is 2.8050690291331852 and acc is 0.3499999940395355
loss on epoch_no. 0 and batch_no 400 is 2.804427985241288 and acc is 0.26250001788139343
loss on epoch_no. 0 and batch_no 500 is 2.8030011725282953 and acc is 0.42500001192092896
loss on epoch_no. 0 and batch_no 600 is 2.801576667934805 and acc is 0.3499999940395355
loss on epoch_no. 0 and batch_no 700 is 2.8050300530802335 and acc is 0.25
loss on epoch_no. 0 and batch_no 800 is 2.8034858391079562 and acc is 0.3499999940395355


KeyboardInterrupt: 

In [24]:
test_losses=[]
test_acc=[]
preds=[]
for idx,batch in enumerate(test_iter):
    with torch.no_grad():
        optimizer.zero_grad()
        uttRNN.eval()
        att.eval()
        convRNN.eval()
        utt_hid=uttRNN(batch.clean_text)
        scores=att(utt_hid,conv_hid.view(batch_size,-1))
        conv_hid,out,hid=convRNN(scores.view(1,1,-1))
        labels=hid2labels(hid.view(batch_size,-1)).to(device)
        preds.append(torch.argmax(labels,dim=1))
        t_loss=loss_fn(labels,batch.act_label_1)
        val_accuracy=get_accuracy(batch.act_label_1,labels)
        test_acc.append(val_accuracy.item())
        test_losses.append(t_loss.item())
#         if(idx%100==0):
#             print('test loss on epoch_no. {} and batch_ no {} is {} and val acc is {}'.format(epoch,idx,np.mean(test_losses),val_accuracy))


RuntimeError: The size of tensor a (10) must match the size of tensor b (80) at non-singleton dimension 0

In [26]:
model=nn.Sequential(uttRNN,att,convRNN,hid2labels)
torch.save(model,'Raheja_model.pth')

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type UtteranceRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Attention. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type ConversationRNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [27]:
l=torch.load('Raheja_model.pth')
print(l)

Sequential(
  (0): UtteranceRNN(
    (gru): GRU(50, 100, bidirectional=True)
  )
  (1): Attention(
    (ws1): Linear(in_features=200, out_features=300, bias=False)
    (ws3): Linear(in_features=100, out_features=300, bias=True)
    (ws2): Linear(in_features=300, out_features=200, bias=False)
    (projection): Linear(in_features=80, out_features=50, bias=True)
  )
  (2): ConversationRNN(
    (gru): GRU(4000, 8000, bidirectional=True)
  )
  (3): Linear(in_features=200, out_features=55, bias=True)
)
